# Setup

In [9]:
# General libraries 
from datetime import date
from datetime import *
import pandas as pd
import numpy as np
import warnings
import logging
import mlflow
import sys
import os

# LP
import pulp

# Directory

In [13]:
# Get the current working directory
script_dir = os.getcwd()

# Check if we are inside the "notebooks" folder and move one level up
if "notebooks" in script_dir:
    parent_dir = os.path.dirname(script_dir)
    os.chdir(parent_dir)

# Verify the change
cwd = os.getcwd()
print("Current Working Directory:", cwd)


Current Working Directory: /Users/ignasipascual/Library/Mobile Documents/com~apple~CloudDocs/Documents


# Assets

In [11]:
# Load assets
from utils.simulator import Simulator
from utils.plotter import Plotter

ModuleNotFoundError: No module named 'utils'

# Parameters

In [4]:
# Set random seed
np.random.seed(42)

# Scenarios
scenario_group = ['plant', 'warehouse', 'market', 'product']

# Supply params
supply_dist='uniform'
supply_params = (200, 50)

# Sell In params
sell_in_dist='normal'
sell_in_params = (150, 40)

# Lead time params
lead_time_dist='uniform'
lead_time_params=(3, 10)

# Initial inventory
initial_inventory = 200

# Start simulator

In [5]:
# Initialize simulator
sim = Simulator()

# Show
sim

## Simulate data

In [6]:
# Example usage:
plants = ['plant_1']
warehouses = ['warehouse_1', 'warehouse_2']
markets = ['market_1', 'market_2']
products = ['product_1']
weeks = [202501 + i for i in range(10)]

# Distribution map
distribution_map = {
    "warehouse_1": ["market_1"], 
    "warehouse_2": ["market_1", "market_2"], 
}

In [7]:
# Generate the dataset
sc_data = sim.generate_scenarios(
    plants, 
    warehouses, 
    markets, 
    products, 
    weeks, 
    distribution_map
)

# Show
sc_data.head()

,plant,warehouse,market,product,week
0,plant_1,warehouse_1,market_1,product_1,202501
3,plant_1,warehouse_1,market_1,product_1,202502
6,plant_1,warehouse_1,market_1,product_1,202503
9,plant_1,warehouse_1,market_1,product_1,202504
12,plant_1,warehouse_1,market_1,product_1,202505


In [8]:
# Printing distinct values
print("Distinct plants:", sc_data['plant'].unique())
print("Distinct warehouses:", sc_data['warehouse'].unique())
print("Distinct markets:", sc_data['market'].unique())
print("Distinct products:", sc_data['product'].unique())

Distinct plants: ['plant_1']
Distinct warehouses: ['warehouse_1' 'warehouse_2']
Distinct markets: ['market_1' 'market_2']
Distinct products: ['product_1']


## Generate scenarios

In [9]:
# Simulate data
sc_data = sim.simulate_flows(
    supply_dist,
    supply_params,
    sell_in_dist,
    sell_in_params
)

# Show
sc_data.head()

,plant,warehouse,market,product,week,supply,sell_in
0,plant_1,warehouse_1,market_1,product_1,202501,144.0,104.0
3,plant_1,warehouse_1,market_1,product_1,202502,57.0,165.0
6,plant_1,warehouse_1,market_1,product_1,202503,90.0,126.0
9,plant_1,warehouse_1,market_1,product_1,202504,110.0,138.0
12,plant_1,warehouse_1,market_1,product_1,202505,177.0,126.0


## Calculate inventory

In [10]:
# Calculate inventory
sc_data_inv = sim.calculate_inventory(
    initial_inventory
)

# Show
sc_data_inv.head()

,plant,warehouse,market,product,week,supply,sell_in,inventory
0,plant_1,warehouse_1,market_1,product_1,202501,144.0,104.0,240.0
3,plant_1,warehouse_1,market_1,product_1,202502,57.0,165.0,132.0
6,plant_1,warehouse_1,market_1,product_1,202503,90.0,126.0,96.0
9,plant_1,warehouse_1,market_1,product_1,202504,110.0,138.0,68.0
12,plant_1,warehouse_1,market_1,product_1,202505,177.0,126.0,119.0


## Lead times

In [11]:
# Add lead times
sc_data_lt = sim.simulate_lead_times(
    scenario_group, 
    lead_time_dist, 
    lead_time_params
)

# Show
sc_data_lt.head()

,plant,warehouse,market,product,week,supply,sell_in,inventory,lead_time
0,plant_1,warehouse_1,market_1,product_1,202501,144.0,104.0,240.0,8
1,plant_1,warehouse_1,market_1,product_1,202502,57.0,165.0,132.0,8
2,plant_1,warehouse_1,market_1,product_1,202503,90.0,126.0,96.0,8
3,plant_1,warehouse_1,market_1,product_1,202504,110.0,138.0,68.0,8
4,plant_1,warehouse_1,market_1,product_1,202505,177.0,126.0,119.0,8


## Summary

In [12]:
# Get summary
summary = sim.get_summary()

# Show
summary

inventory                      supply                        \
                  min    max    mean   std    min    max    mean    std   
warehouse                                                                 
warehouse_1      -4.0  240.0   91.60  69.2   57.0  191.0  122.00  47.52   
warehouse_2     142.0  310.0  231.65  42.2   55.0  197.0  140.25  39.42   

            sell_in                       lead_time                 
                min    max    mean    std       min max mean   std  
warehouse                                                           
warehouse_1   101.0  224.0  142.40  39.14         8   8  8.0  0.00  
warehouse_2    72.0  192.0  142.85  36.64         4   6  5.0  1.03

# Save data

In [13]:
# Write csv
sc_data_lt.to_csv('data/simulated_data.csv', index=False)